# GADM process layers
Our clients want to use their country context, so we have to edit our GADM layer and make these new country context fit into GADM layer
Here is the process:
- Locate the polygon we want to substitute in the GADM layer
- Delete it from the layer
- from the new polygon, get coordinate possitions
- select the surrounding countries
- snap the surrounding countries to the polygon

- import layers:
    * GADM layer
    * Client country 

- Main idea: https://gis.stackexchange.com/questions/245064/snapping-multiple-nodes-of-a-polygon-to-the-nodes-of-a-line
- Create a linestring from the polygon: https://gis.stackexchange.com/questions/290756/transforming-a-polygon-to-a-linestring
- Snap Geopandas: https://gis.stackexchange.com/questions/290092/how-to-do-snapping-in-geopandas
- https://gis.stackexchange.com/questions/203058/why-is-shapelys-snapping-geo-snaps-not-working-as-expected

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Polygon, MultiPolygon, Point, LineString, box
import os
from sys import path
import matplotlib.pyplot as plt
from rtree import index
from scipy.spatial import cKDTree
import itertools
from operator import itemgetter



In [2]:
"""Create dataframes of the import data"""
os.chdir(r"C:\Users\ruben.crespo\Documents\03_tests\administrative_units\09_norway_case")

client_country_name = "NOR"

gadm_path = r"C:\Users\ruben.crespo\Documents\03_tests\administrative_units\09_norway_case\administrative_units_un_gadm_level0_can_rus_3.shp"
client_path = r"C:/Users/ruben.crespo/Documents/03_tests/administrative_units/09_norway_case/kommuner_2022_wgs84_navn.shp"
gadm_gdf = gpd.read_file(gadm_path)
client_gdf = gpd.read_file(client_path)

In [3]:
# https://stackoverflow.com/questions/60780959/how-to-filter-a-geodataframe-by-geometry-type
"""Here we check the geometry type and the number of polygons 6f the import file"""
geometry_dic = {}
geometry_type = client_gdf.geom_type.unique().tolist() #we create a list of unique geometries
if len(geometry_type) > 0: #if there are more than one type we list them
    for i in geometry_type:
        geometry_gdf = client_gdf[client_gdf.geom_type == i] #new df with the geometry
        geometry_rows = geometry_gdf.shape[0] #number of elements
        geometry_dic.update({i:geometry_rows}) #add to dic
        
        
print(geometry_dic)

# geometry_str = ''.join(geometry_gdf.geom_type.unique().tolist()) #transform the list to string


{'Polygon': 363}


In [3]:
"""if we have a multipolygon (lots of polygons), we dissolve it"""
client_area_gdf = client_gdf.dissolve()
# check again its geometry
print(str(client_area_gdf.geom_type))

0    Polygon
dtype: object


In [4]:
"""Here we create the GADM boundaries and the client country"""

"""We extract from GADM the client ccountry"""
client_gadm_gdf = gadm_gdf.loc[(gadm_gdf["CID"] == client_country_name)] 

"""Extract from GADM the surrounding countries of the one we want to edit"""
for row_index, row in gadm_gdf.iterrows():
    if row[0] == client_country_name: #the country we are importing
        neightbour_countries_gdf = gadm_gdf[gadm_gdf.geometry.touches(row['geometry'])] #esta bien
    else:
        pass

"""Union the gadm and client layers and dissolve it"""
gadm_countries_gdf = neightbour_countries_gdf.append(client_gadm_gdf) #esta bien
gadm_boundaries_gdf = gadm_countries_gdf.dissolve() #esta bien

"""Diference of the client and the gadm countries"""
client_country_gs = client_area_gdf.difference(gadm_boundaries_gdf, align= False) #this is the template, esta bien
client_country_gs = client_area_gdf.difference(client_country_gs, align= False) #no esta bien, sale un pico en un extremo
client_country_gdf = gpd.GeoDataFrame(geometry=client_country_gs) #esta bien
#time = 1,69

C:\Users\ruben.crespo\AppData\Local\Temp\ipykernel_11512\628527967.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gadm_countries_gdf = neightbour_countries_gdf.append(client_gadm_gdf)


In [17]:
neightbour_countries_gdf.head()

,CID,Shape_Leng,Shape_Area,Country_n,GID,GID_list,GID_n,geometry
61,FIN,231.567531,62.931585,Finland,FIN,FIN,Finland,"MULTIPOLYGON (((22.49806 59.72403, 22.49806 59..."
182,SWE,352.792209,79.228140,Sweden,SWE,SWE,Sweden,"MULTIPOLYGON (((12.94139 55.46375, 12.94139 55..."
377,RUS,70.305320,30.520940,Russia,RUS45,"RUS1,RUS...,RUS84",Murmansk,"MULTIPOLYGON (((40.53593 67.83385, 40.53750 67..."


In [18]:
finished_countries_gs

61     MULTIPOLYGON (((22.49806 59.72403, 22.49806 59...
182    MULTIPOLYGON (((12.94139 55.46375, 12.94139 55...
377    MULTIPOLYGON (((40.53593 67.83385, 40.53750 67...
dtype: geometry

In [19]:
"""Difference of the neightbours and the client"""
# new_neightbour_countries_gdf = neightbour_countries_gdf.difference(client_country_gdf, align= False) #esta bien

"""We create an empty GeoDataFrame to append inside the country geometries that are finished"""
finished_countries_gs = gpd.GeoSeries()#.reindex_like(new_neightbour_countries_gs)
finished_countries_gs.crs = "EPSG:4326"

countries = []
for row_index, country in neightbour_countries_gdf.iterrows():
    countries.append(country[0])

for i in countries:
    """Get the country"""
    country_gdf = neightbour_countries_gdf.loc[(neightbour_countries_gdf["CID"] == i)] 
    
    """Difference of the country and the client"""
    country_gs = country_gdf.difference(client_country_gdf, align= False) #esta bien
    
    """Append new country to the finished gs list"""
    finished_countries_gs = finished_countries_gs.append(country_gs)

"""Update the neighbours gdf with the gs"""
neightbour_countries_gdf = neightbour_countries_gdf.set_geometry(finished_countries_gs) #works fine

c:\Users\ruben.crespo\Anaconda3\envs\geoprocessing\lib\site-packages\geopandas\geoseries.py:207: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = pd.Series(data, index=index, name=name, **kwargs)
c:\Users\ruben.crespo\Anaconda3\envs\geoprocessing\lib\site-packages\geopandas\geoseries.py:551: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  val = getattr(super(GeoSeries, self), mtd)(*args, **kwargs)
c:\Users\ruben.crespo\Anaconda3\envs\geoprocessing\lib\site-packages\geopandas\geoseries.py:551: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  val = getattr(super(GeoSeries, self), mtd)(*args, **kwargs)
c:\Users\ruben.crespo\Anaconda3\envs\geoprocessing\lib\site-packages\geopandas\geoseries.py:551: Fut

In [22]:
"""Union the client and the neightbours"""
gadm_client_union_gs = client_country_gdf.geometry.union(neightbour_countries_gdf.dissolve(), align=False)

In [23]:
"""Get the remainings"""
remainings_gs = gadm_boundaries_gdf.difference(gadm_client_union_gs, align= False)
# remainings_gs = gadm_union_gs.difference(country_union_gs, align= False)

In [24]:
remainings_gs.to_file('remainings_gs.shp', driver="ESRI Shapefile")

c:\Users\ruben.crespo\Anaconda3\envs\geoprocessing\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [18]:
"""Explode the remainings and create a gdf out of it"""

remainings_gdf = gpd.GeoDataFrame(geometry=remainings_gs)
remainings_gdf = remainings_gdf.explode(index_parts=False)
remainings_gdf = remainings_gdf.reset_index().drop(["level_0", "level_1"], axis = 1) #this is to remove the multiindex

In [19]:
remainings_gdf['centroid'] = remainings_gdf.centroid
# remainings_centroid_gdf = gpd.GeoDataFrame(geometry=remainings_centroid_gs)
remainings_gdf.head()

C:\Users\ruben.crespo\AppData\Local\Temp\ipykernel_13148\3728427008.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  remainings_gdf['centroid'] = remainings_gdf.centroid


In [29]:
for row_index, row_neightbour in neightbour_countries_gdf.iterrows():
    print(type(row_neightbour['geometry']))

<class 'shapely.geometry.multipolygon.MultiPolygon'>
<class 'shapely.geometry.multipolygon.MultiPolygon'>
<class 'shapely.geometry.multipolygon.MultiPolygon'>


In [ ]:
"""Esto pinta bien"""
closest_neightbour_gs = gpd.GeoSeries()#.reindex_like(new_neightbour_countries_gs)
closest_neightbour_gs.crs = "EPSG:4326"

polygon_list = []

for row_index, row_remainings in remainings_gdf.iterrows():
    # print(type(row['centroid'])) #geometry point
    closest_polygon = min(neightbour_countries_gdf['geometry'], key=row_remainings['centroid'].distance) #geometry_type
    for row_index, row_neightbour in neightbour_countries_gdf:
        if closest_polygon == row_neightbour['geometry']:
            
    polygon_list.append(closest_polygon)
    
    
closest_neightbour_gs = gpd.GeoSeries(polygon_list)
closest_neightbour_gs.crs = "EPSG:4326"

remainings_gdf['closest_neigh'] = closest_neightbour_gs #chekea el index porque siempre es 0
# remainings_gdf
# neightbour_countries_gdf

In [21]:
"""Esto pinta bien"""
closest_neightbour_gs = gpd.GeoSeries()#.reindex_like(new_neightbour_countries_gs)
closest_neightbour_gs.crs = "EPSG:4326"

polygon_list = []

for row_index, row_remainings in remainings_gdf.iterrows():
    # print(type(row['centroid'])) #geometry point
    closest_polygon = min(neightbour_countries_gdf['geometry'], key=row_remainings['centroid'].distance) #geometry_type  
    polygon_list.append(closest_polygon)
    
    
closest_neightbour_gs = gpd.GeoSeries(polygon_list)
closest_neightbour_gs.crs = "EPSG:4326"

remainings_gdf['closest_neigh'] = closest_neightbour_gs #chekea el index porque siempre es 0
# remainings_gdf
# neightbour_countries_gdf

c:\Users\ruben.crespo\Anaconda3\envs\geoprocessing\lib\site-packages\geopandas\geoseries.py:207: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = pd.Series(data, index=index, name=name, **kwargs)


In [16]:
closest_neightbour_gs

0       MULTIPOLYGON (((12.94139 55.46375, 12.94139 55...
1       MULTIPOLYGON (((12.94139 55.46375, 12.94139 55...
2       MULTIPOLYGON (((40.53593 67.83385, 40.53750 67...
3       MULTIPOLYGON (((40.53593 67.83385, 40.53750 67...
4       MULTIPOLYGON (((40.53593 67.83385, 40.53750 67...
                              ...                        
1711    MULTIPOLYGON (((12.94139 55.46375, 12.94139 55...
1712    MULTIPOLYGON (((12.94139 55.46375, 12.94139 55...
1713    MULTIPOLYGON (((12.94139 55.46375, 12.94139 55...
1714    MULTIPOLYGON (((12.94139 55.46375, 12.94139 55...
1715    MULTIPOLYGON (((12.94139 55.46375, 12.94139 55...
Length: 1716, dtype: geometry

In [22]:
# remainings_gdf['closest_neigh'] = closest_neightbour_gs
remainings_gdf.head()

,geometry,centroid,closest_neigh
0,"POLYGON ((3.90469 60.18490, 3.90625 60.18542, ...",POINT (3.90625 60.18438),"MULTIPOLYGON (((12.94139 55.46375, 12.94139 55..."
1,"POLYGON ((11.21917 59.08695, 11.22486 59.08761...",POINT (11.22546 59.08733),"MULTIPOLYGON (((12.94139 55.46375, 12.94139 55..."
2,"POLYGON ((30.82240 69.79010, 30.82344 69.79115...",POINT (30.84569 69.77750),"MULTIPOLYGON (((40.53593 67.83385, 40.53750 67..."
3,"POLYGON ((30.87122 69.75460, 30.87404 69.75218...",POINT (30.87149 69.75323),"MULTIPOLYGON (((40.53593 67.83385, 40.53750 67..."
4,"POLYGON ((30.87738 69.75045, 30.87902 69.75035...",POINT (30.88142 69.74205),"MULTIPOLYGON (((40.53593 67.83385, 40.53750 67..."


In [53]:
remainings_gdf.to_file('remainings_gdf.shp', driver="ESRI Shapefile")

In [33]:
#https://www.geeksforgeeks.org/python-map-function/

gpd1 = gpd.GeoDataFrame([['John', 1, Point(1, 1)],
                         ['Smith', 1, Point(2, 2)],
                         ['Soap', 1, Point(0, 2)]],
                        columns=['Name', 'ID', 'geometry'])
gpd2 = gpd.GeoDataFrame([['Work', LineString([Point(100, 0), Point(100, 1)])],
                         ['Shops', LineString([Point(101, 0), Point(101, 1), Point(102, 3)])],
                         ['Home',  LineString([Point(101, 0), Point(102, 1)])]],
                        columns=['Place', 'geometry'])

gdfB_cols=['Place']
A = np.concatenate([np.array(geom.coords) for geom in gpd1.geometry.to_list()]) #we make everything an array of points
B = [np.array(geom.coords) for geom in gpd2.geometry.to_list()] #here we have an array per object
B_ix = tuple(itertools.chain.from_iterable([itertools.repeat(i, x) for i, x in enumerate(list(map(len, B)))])) #we create an index, relating the created point to the object index
B = np.concatenate(B) #we split the array and create only one with everything

In [ ]:

ckd_tree = cKDTree(B)
dist, idx = ckd_tree.query(A, k=1)
idx = itemgetter(*idx)(B_ix)
gdf = pd.concat([gpd1, gpd2.loc[idx, gdfB_cols].reset_index(drop=True), pd.Series(dist, name='dist')], axis=1)

ANother thing

In [24]:
"""We create an empty GeoDataFrame to append inside the country geometries that are finished"""
finished_countries_gs = gpd.GeoSeries()#.reindex_like(new_neightbour_countries_gs)
finished_countries_gs.crs = "EPSG:4326"

c:\Users\ruben.crespo\Anaconda3\envs\geoprocessing\lib\site-packages\geopandas\geoseries.py:207: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = pd.Series(data, index=index, name=name, **kwargs)


In [8]:
"""Get the country list, and start doing a difference with both main and surrounding countries""" 
countries = []
for row_index, country in neightbour_countries_gdf.iterrows():
    countries.append(country[0])
countries

['FIN', 'SWE', 'RUS']

In [ ]:
for i in countries:
    """Get the country"""
    country_gdf = neightbour_countries_gdf.loc[(neightbour_countries_gdf["CID"] == i)] 
    
    """Difference of the country and the client"""
    country_gs = country_gdf.difference(client_country_gdf, align= False) #esta bien
    
    """Union of the country and the client"""
    country_union_gs = country_gs.union(client_country_gdf.geometry, align=False)
    
    country_union_gs.to_file('country_union_gs{}.shp'.format(i), driver="ESRI Shapefile")
    
    #el error esta aqui
    # country_union_gs.to_file('country_union_gs{}.shp'.format(i), driver="ESRI Shapefile")
    
    """Union the gadm ones"""
    gadm_country_gdf = neightbour_countries_gdf.loc[(neightbour_countries_gdf["CID"] == i)]
    gadm_union_gs = client_gadm_gdf.geometry.union(gadm_country_gdf.geometry, align=False)
    
    """Get the remaninings of the difference with both gadm union and the other"""
    remainings_gs = gadm_union_gs.difference(country_union_gs, align= False)
    
    gadm_union_gs=None
    
    """Union the remainings with the country"""
    country_gdf = country_gdf.union(remainings_gs, align= False)
    
    """Append new country to the finished gs list"""
    finished_countries_gs = finished_countries_gs.append(country_gdf.geometry)
    #time = 1,20

In [ ]:
country_gdf = neightbour_countries_gdf.loc[(neightbour_countries_gdf["CID"] == "FIN")] 

"""Difference of the country and the client"""
country_gs = country_gdf.difference(client_country_gdf, align= False) #esta bien

"""Union of the country and the client"""
country_union_gs = country_gs.union(client_country_gdf.geometry, align=False) #vamos a evitar esta union

"""Get the remaninings of the difference with both gadm union and the other"""
remainings_gs = gadm_union_gs.difference(country_union_gs, align= False)
remainings_gs.to_file('remainings_gs.shp', driver="ESRI Shapefile")

"""Split the remainings to the corresponding country"""

remainings_gs.plot()

In [18]:
"""Update the neighbours gdf with the gs"""
neightbour_countries_gdf = neightbour_countries_gdf.set_geometry(finished_countries_gs)
neightbour_countries_gdf

,CID,Shape_Leng,Shape_Area,Country_n,GID,GID_list,GID_n,geometry
61,FIN,231.567531,62.931585,Finland,FIN,FIN,Finland,"MULTIPOLYGON (((22.49806 59.72430, 22.49917 59..."
182,SWE,352.792209,79.228140,Sweden,SWE,SWE,Sweden,"MULTIPOLYGON (((12.94139 55.46292, 12.94195 55..."
377,RUS,70.305320,30.520940,Russia,RUS45,"RUS1,RUS...,RUS84",Murmansk,"MULTIPOLYGON (((40.53750 67.83334, 40.53959 67..."


In [19]:
"""Do the exportation to see wtf is happening"""
neightbour_countries_gdf.to_file('neighbour_new_fixed.shp', driver="ESRI Shapefile")

c:\Users\ruben.crespo\Anaconda3\envs\geoprocessing\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
